# Problem 4 : Estimation - BLP

In [10]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(111);

In [44]:
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));

x = Array(df[!,[:x, :p]]);

# draw a vector of normal shocks
ν = transpose(rand(MvNormal([0;0], [1 0; 0 1]), 20))

Γ = ones(2,2)
Γ[1,2] = 0

δ = ones(size(df, 1))

600-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0

In [47]:
summed_jt_i = exp.(δ .+ x * Γ * ν') # this returns a 600 (jt) by 20 (i) 
sum(summed_jt_i, dims=2) # this sums across individuals 

600×1 Matrix{Float64}:
     57.00582229965489
    233.16347456628938
     62.23148664334593
    139.0192307176588
      1.9572010631035707e13
      7.5191040125367e9
     64.61729047613422
     55.921108756838976
    176.30447614723312
   4600.2410653501875
      1.1154570965762811e6
      4.802172137882022e8
     54.97046030264811
      ⋮
     92.05096050484761
    183.74617118856838
    707.4496820917077
  87825.27102073307
   3984.880690599852
   6384.160433719341
     54.68272891024609
     57.629424848527094
  89979.25568635199
 175360.58854399397
 112097.32688149536
      3.2869182094897693e19

# Part 1: BLP
The moment condition is:


In [6]:
# share
function(δ, Γ)
    exp.(δ + (x' * Γ) .* ν)
    1 + sum(exp.(δ .+ x' * Γ * ν))
end

,market,choice,shares,p,x,z1,z2,z3
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1,0.0099,0.000277852,-0.210973,-6.36336,-5.50945,-5.37102
2,1,2,0.2085,0.000457065,-1.55758,-5.19816,-4.39604,-4.26527
3,1,3,0.009,1.01684,-0.799984,-0.0408355,0.835758,-0.520122
4,1,4,0.0348,0.109472,-1.33837,-1.53891,-1.1727,0.189448
5,1,5,0.3661,5.38779,3.31633,0.758555,-0.70047,0.723864
6,1,6,0.0231,4.18812,2.15453,1.04464,-0.724283,-1.32533
7,2,1,0.0078,0.000566978,0.470776,-5.87798,-4.01759,-5.99733
8,2,2,0.0296,0.000313199,-0.153134,-6.24899,-4.75995,-5.71777
9,2,3,0.1604,0.724284,-2.18476,0.851501,0.426509,1.73536


In [7]:
sort(df, [:x])

,market,choice,shares,p,x,z1,z2,z3
,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,72,1,0.3401,0.000225516,-3.85332,-4.78115,-6.80821,-6.30733
2,92,2,0.3143,0.000132022,-3.41272,-5.60584,-4.70989,-6.505
3,83,2,0.2627,0.000115039,-2.55478,-5.72135,-5.73677,-5.17547
4,43,2,0.2542,8.94e-5,-2.22481,-6.45097,-5.2099,-6.54135
5,2,3,0.1604,0.724284,-2.18476,0.851501,0.426509,1.73536
6,70,3,0.1893,0.298253,-2.10269,0.635707,0.436411,-0.15119
7,21,4,0.1849,0.133983,-2.01491,0.116368,-0.686822,0.608534
8,47,4,0.0123,2.72239,-1.98265,1.66492,-0.0952566,2.23781
9,32,3,0.1902,0.233996,-1.87098,-0.79491,1.96885,1.2329
